In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import random 
import scipy as sc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import stormeunice as eun
import matplotlib.transforms as mtransforms
from cmcrameri import cm

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)

# Import data

In [2]:
lon_min = -15
lon_max = 5
lat_min = 45
lat_max = 60

In [3]:
# ERA5

era5_2022 = xr.open_dataset('/gf3/predict2/AWH012_LEACH_NASTORM/DATA/ERA5/EU025/sfc/2022.nc')
era5_GUST = era5_2022.fg10.sel(time=slice('2022-02-07','2022-02-22'),latitude=slice(lat_max,lat_min),longitude=slice(lon_min, lon_max)).resample(time='3h').max().load()
era5_PRES = era5_2022.msl.sel(time=slice('2022-02-07','2022-02-22'),latitude=slice(lat_max,lat_min),longitude=slice(lon_min, lon_max)).resample(time='3h').max().load()

In [4]:
# EPS
inidates = ['2022-02-14']
experiments = ['pi', 'curr', 'incr']
eps = {}
for inidate in inidates:
    eps[inidate] = eun.data.Data.get_eps_data(experiments, inidate = inidate)

/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/pi/EU025/sfc/cf/b2nq_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/pi/EU025/sfc/pf/b2nq_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/cf/1_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/pf/1_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/incr/EU025/sfc/cf/b2nr_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/incr/EU025/sfc/pf/b2nr_2022-02-10.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/pi/EU025/sfc/cf/b2nn_2022-02-14.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/pi/EU025/sfc/pf/b2nn_2022-02-14.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/cf/1_2022-02-14.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/ENS/EU025/sfc/pf/1_2022-02-14.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/incr/EU025/sfc/cf/b2no_2022-02-14.nc
/gf3/predict2/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/incr/EU025/sfc/pf/b2no_2022-02-14

In [5]:
# analysis
analysis = xr.open_dataset('/home/e/ermis/Storm_Eunice/raw_data/1_2022-02-18_analysis.nc')
analysis_PRES = analysis.msl.sel(time=slice('2022-02-07','2022-02-22'),latitude=slice(lat_max,lat_min),longitude=slice(lon_min, lon_max)).resample(time='3h').max().load()

forecast = xr.open_dataset('/home/e/ermis/Storm_Eunice/raw_data/1_2022-02-18.nc')
forecast_GUST = forecast.fg10.sel(time=slice('2022-02-07','2022-02-22'),latitude=slice(lat_max,lat_min),longitude=slice(lon_min, lon_max)).resample(time='3h').max().load()


# Storm footprints

We define the storm footprints for Eunice to be the maximum 10m 3s wind gust on February 18, 2022. The overall fotprint will then be given by the area where more than or equal to 20m/s were forecast.

In [20]:
# calculate maximum wind gusts on Feb 18
def get_footprint(ds):
    footprint = ds.max(dim='time').copy(deep=True)
    footprint.fg10 = np.zeros(footprint.fg10.shape)
    footprint.fg10 = np.where(ds.fg10.sel > 20, 1, 0)
    return footprint

footprint_curr = eps['2022-02-14']['curr'].fg10.max(dim='time').where(eps['2022-02-14']['curr'].fg10.max(dim='time') > 20, 0)
footprint_incr = eps['2022-02-14']['incr'].fg10.max(dim='time').where(eps['2022-02-14']['incr'].fg10.max(dim='time') > 20, 0)
footprint_pi = eps['2022-02-14']['pi'].fg10.max(dim='time').where(eps['2022-02-14']['pi'].fg10.max(dim='time') > 20, 0)

In [17]:
def calcArea(lat, lon): 
    degToRad = np.pi / 180
    diffLat, diffLon = -np.diff(lat).mean(), np.diff(lon).mean()
    llat, llon = np.meshgrid(lat,lon)
    llatHalf, llonHalf = (llat+diffLat/2)*degToRad, (llon+diffLon/2)*degToRad
    radius = 6371
    areas = radius**2 * np.diff(llonHalf, axis = 0)[:,:-1] * -((np.cos(llatHalf[:,:-1]) - np.cos(llatHalf[:,1:])))[:-1,:]
    return np.transpose(areas)

In [18]:
def maskOcean(dataset, mask):
    maskedDataset = dataset.where(mask.unknown.notnull, drop = False)
    return(maskedDataset)

In [53]:
areas = calcArea(footprint_curr.latitude.values, footprint_curr.longitude.values)
areas = np.vstack((areas, areas[0, :])) # for longitudes at same lat, area is the same
areas = np.hstack((areas, areas[:, 0].reshape(-1, 1))) # assuming a minor change in area for the last latitude

area_curr = [np.sum(areas[footprint_curr.sel(number=x) > 20]) for x in range(51)]
area_incr = [np.sum(areas[footprint_incr.sel(number=x) > 20]) for x in range(51)]
area_pi = [np.sum(areas[footprint_pi.sel(number=x) > 20]) for x in range(51)]

In [58]:
np.mean(area_curr)/1000000, np.mean(area_incr)/1000000, np.mean(area_pi)/1000000

(12.29492645139258, 13.186213374308002, 11.544161394211915)

# Significance

In [65]:
# bootstrap from list of areas and calculate 90% confidence intervals
def bootstrap(data, n=10000, func=np.mean):
    sample = np.random.choice(data, (n, len(data)), replace=True).mean(axis=1)
    conf_int = []
    conf_int.append(func(np.percentile(sample, 5)))
    conf_int.append(func(np.percentile(sample, 95)))
    return conf_int

In [68]:
bootstrap(area_curr), bootstrap(area_incr), bootstrap(area_pi)

([12103963.321842011, 12484403.86170048],
 [12975732.20867008, 13399850.81957744],
 [11361226.379337551, 11726514.19614719])